# Configuring glimpses for UMAP

In [1]:
import napari
from macrohet import dataio, tile
import btrack
print(btrack.__version__)
import os
from tqdm.auto import tqdm
import numpy as np

0.6.1.dev30


### Load experiment of choice

The Opera Phenix is a high-throughput confocal microscope that acquires very large 5-dimensional (TCZXY) images over several fields of view in any one experiment. Therefore, a lazy-loading approach is chosen to mosaic, view and annotate these images. This approach depends upon Dask and DaskFusion. The first step is to load the main metadata file (typically called `Index.idx.xml` and located in the main `Images` directory) that contains the image filenames and associated TCXZY information used to organise the images.

In [2]:
base_dir = '/mnt/DATA/macrohet/'
# base_dir = '/Volumes/lab-gutierrezm/home/users/dayn/macrohet_nemo/'
metadata_fn = os.path.join(base_dir, 'macrohet_images/Index.idx.xml')
metadata = dataio.read_harmony_metadata(metadata_fn)  
metadata_path = os.path.join(base_dir, 'macrohet_images/Assaylayout/20210602_Live_cell_IPSDMGFP_ATB.xml')
assay_layout = dataio.read_harmony_metadata(metadata_path, assay_layout=True,)# mask_exist=True,  image_dir = image_dir, image_metadata = metadata)

Reading metadata XML file...


Extracting HarmonyV5 metadata:   0%|          | 0/113400 [00:00<?, ?it/s]

Extracting metadata complete!
Reading metadata XML file...
Extracting metadata complete!


### Define row and column of choice

In [3]:
row = 3
column = 5

### Now to lazily mosaic the images using Dask prior to viewing them.

1x (75,2,3) [TCZ] image stack takes approximately 1 minute to stitch together, so only load the one field of view I want.

In [90]:
import importlib
importlib.reload(tile)

<module 'macrohet.tile' from '/home/dayn/analysis/macrohet/macrohet/tile.py'>

In [92]:
image_dir = os.path.join(base_dir, 'macrohet_images/Images')
images = tile.compile_mosaic(image_dir, 
                             metadata, 
                             row, column, 
                             set_plane='sum_proj',
#                              set_channel=1,
                             set_time = 0,
#                             input_transforms = [input_transforms]
                            )#.compute().compute()

In [93]:
images

dask.array<astype, shape=(1, 2, 6048, 6048), dtype=uint16, chunksize=(1, 1, 2016, 2016), chunktype=numpy.ndarray>

In [94]:
images = images.compute().compute()

In [96]:
#sum proj 8bit new method
images

array([[[[   0,    0,    0, ...,    0,    0,    0],
         [ 319,  316,  310, ...,  308,  317,    0],
         [ 321,  324,  315, ...,  312,  318,    0],
         ...,
         [1692, 1762, 1880, ..., 1292, 1265,    0],
         [1469, 1580, 1799, ..., 1338, 1232,    0],
         [1350, 1475, 1667, ..., 1365, 1301,    0]],

        [[   0,    0,    0, ...,    0,    0,    0],
         [ 313,  303,  304, ...,  296,  301,    0],
         [ 314,  316,  309, ...,  317,  311,    0],
         ...,
         [ 332,  337,  328, ...,  323,  315,    0],
         [ 335,  342,  318, ...,  325,  320,    0],
         [ 334,  333,  299, ...,  312,  322,    0]]]], dtype=uint16)

In [62]:
#sum proj 8bit old method
images

array([[[[  0,   0,   0, ...,   0,   0,   0],
         [ 20,  20,  19, ...,  19,  20,   0],
         [ 21,  21,  20, ...,  21,  21,   0],
         ...,
         [121, 125, 133, ...,  71,  70,   0],
         [105, 112, 127, ...,  74,  68,   0],
         [ 97, 105, 118, ...,  75,  71,   0]],

        [[  0,   0,   0, ...,   0,   0,   0],
         [ 29,  27,  27, ...,  23,  23,   0],
         [ 29,  30,  28, ...,  25,  23,   0],
         ...,
         [ 23,  24,  22, ...,  19,  18,   0],
         [ 23,  24,  22, ...,  19,  19,   0],
         [ 24,  24,  21, ...,  18,  19,   0]]]], dtype=uint8)

In [55]:
images

array([[[[   0,    0,    0, ...,    0,    0,    0],
         [ 319,  316,  310, ...,  308,  317,    0],
         [ 321,  324,  315, ...,  312,  318,    0],
         ...,
         [1692, 1762, 1880, ..., 1292, 1265,    0],
         [1469, 1580, 1799, ..., 1338, 1232,    0],
         [1350, 1475, 1667, ..., 1365, 1301,    0]],

        [[   0,    0,    0, ...,    0,    0,    0],
         [ 313,  303,  304, ...,  296,  301,    0],
         [ 314,  316,  309, ...,  317,  311,    0],
         ...,
         [ 332,  337,  328, ...,  323,  315,    0],
         [ 335,  342,  318, ...,  325,  320,    0],
         [ 334,  333,  299, ...,  312,  322,    0]]]], dtype=uint16)

In [88]:
napari.Viewer().add_image(images)

/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari_tools_menu/__init__.py:194: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


<Image layer 'images' at 0x7f94fd92b130>

In [83]:
(images[0,0,0,...]+images[0,0,1,...]+images[0,0,2,...])

array([[  0,   0,   0, ...,   0,   0,   0],
       [ 20,  20,  19, ...,  19,  20,   0],
       [ 21,  21,  20, ...,  21,  21,   0],
       ...,
       [121, 125, 133, ...,  71,  70,   0],
       [105, 112, 127, ...,  74,  68,   0],
       [ 97, 105, 118, ...,  75,  71,   0]], dtype=uint8)

In [39]:
np.sum(images, axis=2, dtype='uint8')

array([[[[  0,   0,   0, ...,   0,   0,   0],
         [ 20,  20,  19, ...,  19,  20,   0],
         [ 21,  21,  20, ...,  21,  21,   0],
         ...,
         [121, 125, 133, ...,  71,  70,   0],
         [105, 112, 127, ...,  74,  68,   0],
         [ 97, 105, 118, ...,  75,  71,   0]],

        [[  0,   0,   0, ...,   0,   0,   0],
         [ 29,  27,  27, ...,  23,  23,   0],
         [ 29,  30,  28, ...,  25,  23,   0],
         ...,
         [ 23,  24,  22, ...,  19,  18,   0],
         [ 23,  24,  22, ...,  19,  19,   0],
         [ 24,  24,  21, ...,  18,  19,   0]]]], dtype=uint8)

In [42]:
np.sum(images, axis=2, dtype='u1')

array([[[[  0,   0,   0, ...,   0,   0,   0],
         [ 20,  20,  19, ...,  19,  20,   0],
         [ 21,  21,  20, ...,  21,  21,   0],
         ...,
         [121, 125, 133, ...,  71,  70,   0],
         [105, 112, 127, ...,  74,  68,   0],
         [ 97, 105, 118, ...,  75,  71,   0]],

        [[  0,   0,   0, ...,   0,   0,   0],
         [ 29,  27,  27, ...,  23,  23,   0],
         [ 29,  30,  28, ...,  25,  23,   0],
         ...,
         [ 23,  24,  22, ...,  19,  18,   0],
         [ 23,  24,  22, ...,  19,  19,   0],
         [ 24,  24,  21, ...,  18,  19,   0]]]], dtype=uint8)

In [46]:
np.sum(images, axis=2,)

array([[[[  0,   0,   0, ...,   0,   0,   0],
         [ 20,  20,  19, ...,  19,  20,   0],
         [ 21,  21,  20, ...,  21,  21,   0],
         ...,
         [121, 125, 133, ...,  71,  70,   0],
         [105, 112, 127, ...,  74,  68,   0],
         [ 97, 105, 118, ...,  75,  71,   0]],

        [[  0,   0,   0, ...,   0,   0,   0],
         [ 29,  27,  27, ...,  23,  23,   0],
         [ 29,  30,  28, ...,  25,  23,   0],
         ...,
         [ 23,  24,  22, ...,  19,  18,   0],
         [ 23,  24,  22, ...,  19,  19,   0],
         [ 24,  24,  21, ...,  18,  19,   0]]]], dtype=uint64)

In [38]:
 np.max(images, axis=2).shape

(1, 2, 6048, 6048)

In [16]:
# viewer = napari.Viewer()
viewer.add_image(images, 
                 channel_axis=1,
                 name=["macrophage", "mtb"],
                 colormap=["green",  "magenta"],
#                  contrast_limits=[[100, 6000], [100, 2000]],
                 contrast_limits=[[0,450], [0,450]], 
#                  visible = False
                 )

[<Image layer 'macrophage [3]' at 0x7f4a9ceb8be0>,
 <Image layer 'mtb [3]' at 0x7f4a9d034640>]

In [9]:
images

dask.array<sum-aggregate, shape=(1, 2, 6048, 6048), dtype=uint64, chunksize=(1, 1, 2016, 2016), chunktype=numpy.ndarray>

In [46]:
%%time
images = images.compute().compute().astype(np.u)

CPU times: user 2min 16s, sys: 13.2 s, total: 2min 30s
Wall time: 7.25 s


In [63]:
images

dask.array<astype, shape=(1, 2, 6048, 6048), dtype=uint8, chunksize=(1, 1, 2016, 2016), chunktype=numpy.ndarray>

In [47]:
images.shape

(1, 2, 6048, 6048)

In [64]:
gfp = images[:,0,...]

In [65]:
gfp

dask.array<getitem, shape=(1, 6048, 6048), dtype=uint8, chunksize=(1, 2016, 2016), chunktype=numpy.ndarray>

In [71]:
rfp = images[:,1,...]

# Load tracks

In [5]:
tracks_fn = os.path.join(base_dir, f'labels/macrohet_seg_model/{row, column}.h5')
with btrack.io.HDF5FileHandler(tracks_fn, 'r') as hdf:
    tracks = hdf.tracks
    segmentation = hdf.segmentation
    objects = hdf.objects
napari_tracks, properties, graph = btrack.utils.tracks_to_napari(tracks, ndim=2)

[INFO][2023/05/17 12:59:08 PM] Opening HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(3, 5).h5...
[INFO][2023/05/17 12:59:08 PM] Loading tracks/obj_type_1
[INFO][2023/05/17 12:59:08 PM] Loading LBEP/obj_type_1
[INFO][2023/05/17 12:59:08 PM] Loading objects/obj_type_1 (41868, 5) (41868 filtered: None)
[INFO][2023/05/17 12:59:22 PM] Loading segmentation (75, 6048, 6048)
[INFO][2023/05/17 12:59:22 PM] Loading objects/obj_type_1 (41868, 5) (41868 filtered: None)
[INFO][2023/05/17 12:59:22 PM] Closing HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(3, 5).h5


In [6]:
segmentation.shape

(75, 6048, 6048)

In [7]:
masks = segmentation[0]

# Make a corpus of glimpses

Need to couple an object (ie a track at t) to the segmentation... the question is, how to store that glimpse info? If I store a glimpse within an object item then it keeps the other info too...

In [8]:
scale_factor = 5.04

In [9]:
objects_t0 = [o for o in objects if o.t ==0]

In [11]:
len(objects_t0)

640

### Generating an actual corpus

In [15]:
obj = objects_t0[0]

In [16]:
obj

,ID,x,y,z,t,dummy,states,label,minor_axis_length,mean_intensity,area,Infected,orientation,major_axis_length
0,0,743.478333,86.112717,0.0,0,False,7,5,79.861702,"(3,) array",6685.0,1.0,0.524175,107.259262


In [17]:
import sys

In [ ]:
miport

In [18]:
sys.getsizeof(obj)

128

In [40]:
gfp

array([[  0,   0,   0, ...,   0,   0,   0],
       [ 20,  20,  19, ...,  19,  20,   0],
       [ 21,  21,  20, ...,  21,  21,   0],
       ...,
       [121, 125, 133, ...,  71,  70,   0],
       [105, 112, 127, ...,  74,  68,   0],
       [ 97, 105, 118, ...,  75,  71,   0]], dtype=uint64)

In [37]:
t = 0
masks = segmentation[t]
gfp = images[t,0,...].compute().compute()
rfp = images[t,1,...].compute().compute()
for obj in tqdm(objects_t0):
    coords = int(obj.y*scale_factor), int(obj.x*scale_factor)
    t = obj.t
#     masks = segmentation[t]
#     gfp = images[t,0,...].compute().compute()
#     rfp = images[t,1,...].compute().compute()
    segment_ID = masks[coords]
#     obj.properties['mask'] = masks == [segment_ID]
    segment = masks == [segment_ID]
    gfp_glimpse = segment * gfp
    obj.properties['gfp_glimpse'] = ((masks == [segment_ID])*gfp).astype(np.uint16)
#     obj.properties['rfp_glimpse'] = (obj.properties['mask']*rfp > 480).astype(np.uint16)
    break

  0%|          | 0/640 [00:00<?, ?it/s]

In [ ]:
75*500

In [32]:
sys.getsizeof(obj)

128

In [39]:
sys.getsizeof(obj.properties['gfp_glimpse'])/(1024 * 1024)

69.7677001953125

In [33]:
sys.getsizeof(obj.properties['gfp_glimpse'])/(1024 * 1024)

69.7677001953125

In [29]:
sys.getsizeof(obj.properties['gfp_glimpse'])/(1024 * 1024)

34.8839111328125

In [24]:
sys.getsizeof(obj.properties['gfp_glimpse'])/(1024 * 1024)

279.0704345703125

In [66]:
v = napari.Viewer()
v.add_image(obj.properties['gfp_glimpse'])
v.add_image(obj.properties['rfp_glimpse'])
v.add_image(gfp)
v.add_image(rfp)

/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari_tools_menu/__init__.py:194: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


<Image layer 'rfp' at 0x7f834ccf4340>

### Rename properties? Add mask ID as property?

Issue is that once delted from placeholder it is removed from the original objects list

In [18]:
# define features to use for tracking 
features = [
  "area",
  "major_axis_length",
  "minor_axis_length",
  "orientation",
  "mean_intensity",
  "intensity_image"
    ]
def localise(masks, intensity_image, properties=tuple(features), use_weighted_centroid = False):
    
    # localise objs in images
    objects = btrack.utils.segmentation_to_objects(segmentation=masks,
                                                   intensity_image=intensity_image, 
                                                   properties=properties,
                                                   scale=(scale_factor,scale_factor),
                                                   use_weighted_centroid=use_weighted_centroid, 
                                                   )
                                                   
    return objects


In [33]:
# create pseduo mean intensity with third channel as mask channel
obj1.properties['mean_intensity']

925.25555

In [28]:
obj1.properties

{'minor_axis_length': 81.01919,
 'area': 20624.0,
 'mean_intensity': 925.25555,
 'orientation': -0.10882337,
 'major_axis_length': 334.28482}

In [25]:
obj1.properties['mask_ID'] = obj1.properties['mean_intensity'][-1]
obj1.properties['mean_intensity'] = obj1.properties['mean_intensity'][0:2]

In [26]:
obj1.properties['mean_intensity']

925.25555

# Launch napari image viewer

In [52]:
viewer = napari.Viewer()

viewer.add_image(images, 
                 channel_axis=1,
                 name=["macrophage", "mtb"],
                 colormap=["green",  "magenta"],
#                  contrast_limits=[[100, 6000], [100, 2000]],
                 contrast_limits=[[0,450], [0,450]], 
                 visible = False
                 )
viewer.add_labels(masks, 
#                   num_colors= 1,
                  #scale=(10, 1, 1,), 
                  #color='yellow'
                  name = 'new segmentation'
                 )
# viewer.add_labels(col_segmentation, 
#                   name = 'recolored new segmentation'
#                   #scale=(10, 1, 1,), 
#                   #color='yellow'
#                  )
# viewer.add_tracks(napari_tracks,
# #                     properties=properties, 
# #                     graph=graph, 
#                     name="New model tracks", 
#                     blending="translucent",
#                     visible=True,
#                     scale = (1,5.04,5.04)
#                  )
# viewer.add_tracks(filtered_napari_tracks,
# #                     properties=properties, 
# #                     graph=graph, 
#                     name="New model tracks", 
#                     blending="translucent",
#                     visible=True,
#                     scale = (1,5.04,5.04)
#                  )
# viewer.add_tracks(ds_napari_tracks,
# #                     properties=properties, 
# #                     graph=graph, 
#                     name="downscaled tracks", 
#                     blending="translucent",
#                     visible=True,
# #                     scale = (1,1,1)
#                 )
# viewer.add_labels(ds_segmentation, 
#                  name = 'downscaled segmentation')

# viewer.add_labels(ds_col_segmentation, 
#                   name = 'downscaled coloured segmentation',
#                   scale=(1, 1, 1,), 
# #                   color={1:'yellow'}
#                  )


/home/dayn/analysis/btrack/btrack/dataio.py:3: UserWarning: `btrack.dataio` has been deprecated. Please use `btrack.io` subpackage instead.
  warnings.warn(  # noqa: B028


Assistant skips harvesting pyclesperanto as it's not installed.


/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari_tools_menu/__init__.py:194: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


<Labels layer 'new segmentation' at 0x7fb326c0a0d0>